## Data preparing

In [1]:
import os

current_path = os.path.abspath(os.getcwd())
data_path = current_path + '/train'
csv_path = current_path + '/train.csv'
benign = current_path + '/benign'
malignant = current_path + '/malignant'

In [2]:
files = os.listdir(data_path)


print(len(files))

10983


In [3]:
import csv
import numpy as np
data_csv = []
with open(csv_path, 'rt') as f:
    data = csv.reader(f)
    data_csv = [i for i in data]
 

In [4]:
data_np = np.asarray(data_csv[1:])

error = []

for i in files:
    if(i[-4:] != '.jpg'):
        continue
    file_name = i[:-4]

    name_index = np.where(data_np[:,0] == file_name)

    if(data_np[name_index,6][0,0] == 'benign'):
        source_path = data_path + '/' + i
        destination_path = benign + '/' + i 
        os.rename(source_path,destination_path)

    elif(data_np[name_index,6][0,0] == 'malignant'):
        source_path = data_path + '/' + i
        destination_path = malignant + '/' + i 
        os.rename(source_path,destination_path)    
    else:
        error.append(file_name)

print(error)
        

IndexError: index 6 is out of bounds for axis 1 with size 5

## Tensorflow

In [5]:
import warnings
warnings.filterwarnings('ignore')
import os
import matplotlib
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Nadam
from keras.utils import np_utils,get_file

K.image_data_format() == 'channels_last'

True

In [24]:


def image_resize(image, size):
    new_image = cv.resize(image, siez, interpolation=cv.INTER_AREA)
    return new_image



In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def processing_data(data_path, height, width, batch_size=32, test_split=0.3):


    train_data = ImageDataGenerator(

            rescale=1. / 255,  
            shear_range=0.15,  
            zoom_range=0.15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True,
            vertical_flip=True,
            validation_split=test_split  
    )


    test_data = ImageDataGenerator(
            rescale=1. / 255,
            validation_split=test_split)

    train_generator = train_data.flow_from_directory(

            data_path, 

            target_size=(height, width),
            batch_size=batch_size,
            class_mode='binary',
            classes=['benign','malignant'],
            subset='training', 
            seed=0)
    test_generator = test_data.flow_from_directory(
            data_path,
            target_size=(height, width),
            batch_size=batch_size,
            class_mode='binary',
            classes=['benign','malignant'],
            subset='validation',
            seed=0)

    return train_generator, test_generator

In [26]:
def save_model(model, checkpoint_save_path, model_dir):
    """
    保存模型，每迭代3次保存一次
    :param model: 训练的模型
    :param checkpoint_save_path: 加载历史模型
    :param model_dir: 
    :return: 
    """
    if os.path.exists(checkpoint_save_path):
        print("loading model")
        model.load_weights(checkpoint_save_path)
        print("loading is finished")
    checkpoint_period = ModelCheckpoint(
        # 模型存储路径
        model_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
        # 检测的指标
        monitor='acc',
        # ‘auto’，‘min’，‘max’中选择
        #mode='max',
        # 是否只存储模型权重
        save_weights_only=False,
        # 是否只保存最优的模型
        save_best_only=True,
        # 检测的轮数是每隔2轮
        period=3
    )
    return checkpoint_period

In [27]:
from keras import models
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

def model():
    model = models.Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64,(3,3),activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [28]:

model_name = 'mobilenet_1_0_224_tf_no_top.h5'
weights_path = current_path + '/weight/' + model_name


model = model()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])
print(weights_path)
model.load_weights(weights_path,by_name=True)
checkpoint_save_path = "./results/temp.h5"
model_dir = "./results/"
checkpoint_period = save_model(model, checkpoint_save_path, model_dir)

d:\ece431/weight/mobilenet_1_0_224_tf_no_top.h5


In [34]:
reduce_lr = ReduceLROnPlateau(
                        monitor='acc',  # 检测的指标
                        factor=0.5,     # 当acc不下降时将学习率下调的比例
                        patience=3,     # 检测轮数是每隔两轮
                        verbose=1       # 信息展示模式
                    )
early_stopping = EarlyStopping(
                            monitor='val_loss',  # 检测的指标
                            min_delta=0,         # 增大或减小的阈值
                            patience=10,         # 检测的轮数频率
                            verbose=2           # 信息展示的模式
                        )

In [33]:


# 一次的训练集大小
batch_size = 5
# 图片数据路径

# 图片处理
train_generator,test_generator = processing_data(data_path, height=160, width=160, batch_size=batch_size, test_split=0.1)
# 编译模型
model.compile(loss='binary_crossentropy',  # 二分类损失函数   
              optimizer= Adam(lr=0.001),            # 优化器
              metrics=['accuracy'])        # 优化目标
# 训练模型
history = model.fit_generator(train_generator,    
                    epochs=4, # epochs: 整数，数据的迭代总轮数。
                    # 一个epoch包含的步数,通常应该等于你的数据集的样本数量除以批量大小。
                    steps_per_epoch= 7143// batch_size,
                    validation_data=test_generator,
                    validation_steps=max(1, 792//batch_size),
                    initial_epoch=0, # 整数。开始训练的轮次（有助于恢复之前的训练）。
                    callbacks=[checkpoint_period, reduce_lr])
# 保存模型
model.save_weights(model_dir + 'temp.h5')

Found 7143 images belonging to 2 classes.
Found 792 images belonging to 2 classes.


MemoryError: 